## CS 155 Project 1 Modelling Shenanigans!
Let's start modelling for fun! :)

In [377]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import fbeta_score, make_scorer
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.calibration import CalibratedClassifierCV
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [354]:
def f2(yhat, testy):
    tp = 0
    fp = 0
    fn = 0
    
    for i in range(0,len(yhat)):
        if yhat[i]==1 and testy[i] ==1:
            tp += 1
        if yhat[i]==1 and testy[i] !=1:
            fp += 1
        if yhat[i]==0 and testy[i] !=0:
            fn += 1
    
    p=tp/(tp+fp)
    r=(tp)/(tp+fn)
    
    print('true positives ' + str(tp))
    print('false positives ' + str(fp))
    print('precision ' + str(p))
    print('recall ' + str(r))
    
    print('F2 ' + str(5*p*r/(4*p+r)))

In [355]:
df = pd.read_csv("trainingData2.csv")
df = df.sample(frac=1)

trainsize = int(len(df.index)*0.75)
df_train = df.iloc[:trainsize,:]
df_test = df.iloc[trainsize:,:]
weights = (((df_train['lab']==1)+0.1).values)*6.9
labtest = df_test[df_test['lab']==1.0]

df_train=df_train.drop('lab', axis=1)
df_test=df_test.drop('lab', axis=1)
labtest=labtest.drop('lab', axis=1)
df = df.drop('lab',axis=1)

In [356]:
# Normalizing data
NPTX = df_train.drop('label', axis=1).values
scaler = StandardScaler()
scaler.fit(NPTX)
NPTX=scaler.transform(NPTX)
NPTY = df_test.drop('label', axis=1).values
NPTY=scaler.transform(NPTY)
NPTL = labtest.drop('label', axis=1).values
NPTL=scaler.transform(NPTL)

normalizedTrainDF= pd.DataFrame(NPTX, columns = df_train.loc[:, df.columns != 'label'].columns.values)
normalizedTestDF= pd.DataFrame(NPTY, columns = df_test.loc[:, df.columns != 'label'].columns.values)
normalizedTestLabDF= pd.DataFrame(NPTL, columns = df_test.loc[:, df.columns != 'label'].columns.values)

## Let's first investigate Multicollinearity

In [378]:
vif_data = pd.DataFrame()
vif_data["feature"] = normalizedTrainDF.columns
vif_data["VIF"] = [variance_inflation_factor(normalizedTrainDF.values, i) for i in range(len(normalizedTrainDF.columns))]

In [379]:
vif_data

,feature,VIF
0,meanStepDisplacement,38.598437
1,meanStepTravel,66.790479
2,sdStepDisplacement,225.968565
3,sdStepTravel,118.298189
4,shapiroDisplacement,1.124261
5,meanAngle,3.234422
6,sdAngle,3.395903
7,meanVideoStepDisplacement,21.066899
8,sdVideoMeanStepDisplacement,14.078895
9,meanVideoStepTravel,16.695591


In [382]:
normalizedTrainDF = normalizedTrainDF.drop('sdStepDisplacement', axis=1)
vif_data = pd.DataFrame()
vif_data["feature"] = normalizedTrainDF.columns
vif_data["VIF"] = [variance_inflation_factor(normalizedTrainDF.values, i) for i in range(len(normalizedTrainDF.columns))]
vif_data

,feature,VIF
0,meanStepDisplacement,6.287087
1,meanStepTravel,10.724969
2,sdStepTravel,2.206422
3,shapiroDisplacement,1.113192
4,meanAngle,3.080474
5,sdAngle,2.880853
6,meanVideoStepDisplacement,21.021566
7,sdVideoMeanStepDisplacement,13.960361
8,meanVideoStepTravel,16.685483
9,sdVideoMeanStepTravel,15.717489


In [383]:
normalizedTrainDF = normalizedTrainDF.drop('meanVideoStepDisplacement', axis=1)
vif_data = pd.DataFrame()
vif_data["feature"] = normalizedTrainDF.columns
vif_data["VIF"] = [variance_inflation_factor(normalizedTrainDF.values, i) for i in range(len(normalizedTrainDF.columns))]
vif_data

,feature,VIF
0,meanStepDisplacement,6.174687
1,meanStepTravel,10.635621
2,sdStepTravel,2.203364
3,shapiroDisplacement,1.112698
4,meanAngle,3.007225
5,sdAngle,2.845311
6,sdVideoMeanStepDisplacement,13.898860
7,meanVideoStepTravel,2.455197
8,sdVideoMeanStepTravel,13.961498
9,meanVideoAngle,1.539214


In [384]:
normalizedTrainDF = normalizedTrainDF.drop('sdVideoMeanStepTravel', axis=1)
vif_data = pd.DataFrame()
vif_data["feature"] = normalizedTrainDF.columns
vif_data["VIF"] = [variance_inflation_factor(normalizedTrainDF.values, i) for i in range(len(normalizedTrainDF.columns))]
vif_data

,feature,VIF
0,meanStepDisplacement,6.162006
1,meanStepTravel,10.629771
2,sdStepTravel,2.203220
3,shapiroDisplacement,1.109303
4,meanAngle,2.997273
5,sdAngle,2.842960
6,sdVideoMeanStepDisplacement,2.660547
7,meanVideoStepTravel,2.451670
8,meanVideoAngle,1.527078
9,sdVideoAngle,1.231500


In [386]:
normalizedTrainDF = normalizedTrainDF.drop('meanStepTravel', axis=1)
vif_data = pd.DataFrame()
vif_data["feature"] = normalizedTrainDF.columns
vif_data["VIF"] = [variance_inflation_factor(normalizedTrainDF.values, i) for i in range(len(normalizedTrainDF.columns))]
vif_data

,feature,VIF
0,meanStepDisplacement,1.585585
1,sdStepTravel,1.909319
2,shapiroDisplacement,1.089791
3,meanAngle,2.223566
4,sdAngle,2.181081
5,sdVideoMeanStepDisplacement,2.660455
6,meanVideoStepTravel,2.352688
7,meanVideoAngle,1.489185
8,sdVideoAngle,1.231496


So we cut:

meanStepTravel

sdVideoMeanStepTravel

meanVideoStepDisplacement

sdStepDisplacement

In [396]:
normalizedTestDF = normalizedTestDF.drop(['meanStepTravel','sdVideoMeanStepTravel','meanVideoStepDisplacement',
                                         'sdStepDisplacement'], axis=1)
normalizedTestLabDF = normalizedTestLabDF.drop(['meanStepTravel','sdVideoMeanStepTravel','meanVideoStepDisplacement',
                                         'sdStepDisplacement'], axis=1)

### Stepwise Logistic Regression

In [440]:
ftwo_scorer = make_scorer(fbeta_score, beta=2)

#Stepwise log reg?
sfs = SequentialFeatureSelector(linear_model.LogisticRegression(max_iter=12000),
                                k_features='best',
                                forward=True,
                                scoring='roc_auc',
                                cv=KFold(n_splits=5, shuffle=True,random_state=False))
selected_features = sfs.fit(normalizedTrainDF, df_train['label'].values).k_feature_names_

model = LogisticRegression(multi_class='multinomial', solver='lbfgs',max_iter=12000, class_weight={0:3,1:1})
model.fit(normalizedTrainDF[np.asarray(selected_features)], df_train['label'].values, sample_weight = weights)

# Check accuracy on testing data
yhat = model.predict(normalizedTestDF[np.asarray(selected_features)])
testy= df_test['label'].values
yhat0 = model.predict(normalizedTrainDF[np.asarray(selected_features)])
trainy = df_train['label'].values

yhat2 = model.predict(normalizedTestLabDF[np.asarray(selected_features)])
testy2= labtest['label'].values

print('Train Accuracy: '+ str(np.sum(yhat0==trainy)/len(trainy)))
print( fbeta_score(trainy, yhat0, average='weighted', beta=2))

print('Test Accuracy: '+ str(np.sum(yhat==testy)/len(testy)))
print(fbeta_score(testy, yhat, average='weighted', beta=2))

print('Lab Test Accuracy: '+ str(np.sum(yhat==testy)/len(testy)))
f2(testy2, yhat2)
print()
print('similarity with best model')
vsbest(model)
print()
print(selected_features)

Train Accuracy: 0.9816749585406301
0.9816279517601076
Test Accuracy: 0.9798507462686568
0.9798189557312164
Lab Test Accuracy: 0.9798507462686568
true positives 15
false positives 13
precision 0.5357142857142857
recall 0.7894736842105263
F2 0.7211538461538461

similarity with best model
0.80083857442348

('sdStepTravel', 'shapiroDisplacement', 'meanAngle', 'sdVideoMeanStepDisplacement', 'meanVideoStepTravel', 'meanVideoAngle')


In [439]:
sfs = SequentialFeatureSelector(linear_model.LogisticRegression(max_iter=12000),
                                k_features='parsimonious',
                                forward=True,
                                scoring='roc_auc',
                                cv=KFold(n_splits=10, shuffle=True,random_state=False))
selected_features = sfs.fit(normalizedTrainDF, df_train['label'].values).k_feature_names_

model = LogisticRegression(multi_class='multinomial', solver='lbfgs',max_iter=12000, class_weight={0:2.5,1:1})
model.fit(normalizedTrainDF[np.asarray(selected_features)], df_train['label'].values, sample_weight = weights)

# Check accuracy on testing data
yhat = model.predict(normalizedTestDF[np.asarray(selected_features)])
testy= df_test['label'].values
yhat0 = model.predict(normalizedTrainDF[np.asarray(selected_features)])
trainy = df_train['label'].values

yhat2 = model.predict(normalizedTestLabDF[np.asarray(selected_features)])
testy2= labtest['label'].values

print('Train Accuracy: '+ str(np.sum(yhat0==trainy)/len(trainy)))
print( fbeta_score(trainy, yhat0, average='weighted', beta=2))

print('Test Accuracy: '+ str(np.sum(yhat==testy)/len(testy)))
print(fbeta_score(testy, yhat, average='weighted', beta=2))

print('Lab Test Accuracy: '+ str(np.sum(yhat==testy)/len(testy)))
f2(testy2, yhat2)

print()
print('similarity with best model')
vsbest(model)
print()

print(selected_features)

Train Accuracy: 0.9820066334991708
0.9819702031161787
Test Accuracy: 0.9825870646766169
0.9825666876536896
Lab Test Accuracy: 0.9825870646766169
true positives 15
false positives 13
precision 0.5357142857142857
recall 0.75
F2 0.6944444444444444

similarity with best model
0.8050314465408805

('sdStepTravel', 'shapiroDisplacement', 'meanAngle', 'meanVideoStepTravel', 'meanVideoAngle')


In [339]:
sfs = SequentialFeatureSelector(linear_model.LogisticRegression(max_iter=12000),
                                k_features='best',
                                forward=False,
                                scoring=ftwo_scorer,
                                cv=KFold(n_splits=5, shuffle=True,random_state=False))
selected_features = sfs.fit(normalizedTrainDF, df_train['label'].values).k_feature_names_

model = LogisticRegression(multi_class='multinomial', solver='lbfgs',max_iter=12000, class_weight={0:3,1:1})
model.fit(normalizedTrainDF[np.asarray(selected_features)], df_train['label'].values, sample_weight = weights)

yhat = model.predict(normalizedTestDF[np.asarray(selected_features)])
testy= df_test['label'].values
yhat0 = model.predict(normalizedTrainDF[np.asarray(selected_features)])
trainy = df_train['label'].values


# Check accuracy on testing data
yhat = model.predict(normalizedTestDF[np.asarray(selected_features)])
testy= df_test['label'].values
yhat0 = model.predict(normalizedTrainDF[np.asarray(selected_features)])
trainy = df_train['label'].values

yhat2 = model.predict(normalizedTestLabDF[np.asarray(selected_features)])
testy2= labtest['label'].values

print('Train Accuracy: '+ str(np.sum(yhat0==trainy)/len(trainy)))
print( fbeta_score(trainy, yhat0, average='weighted', beta=2))

print('Test Accuracy: '+ str(np.sum(yhat==testy)/len(testy)))
print(fbeta_score(testy, yhat, average='weighted', beta=2))

print('Lab Test Accuracy: '+ str(np.sum(yhat==testy)/len(testy)))
f2(testy2, yhat2)

print()
print('similarity with best model')
vsbest(model)
print()

print(selected_features)

Train Accuracy: 0.983665008291874
0.9836147646081904
Test Accuracy: 0.9865671641791045
0.9865257345048414
Lab Test Accuracy: 0.9865671641791045
true positives 20
false positives 4
precision 0.8333333333333334
recall 1.0
F2 0.9615384615384615

similarity with best model
0.8029350104821803

('meanStepDisplacement', 'meanStepTravel', 'sdStepDisplacement', 'sdStepTravel', 'shapiroDisplacement', 'meanAngle', 'sdAngle', 'meanVideoStepDisplacement', 'meanVideoStepTravel', 'sdVideoMeanStepTravel', 'meanVideoAngle', 'sdVideoAngle')


In [340]:
sfs = SequentialFeatureSelector(linear_model.LogisticRegression(max_iter=12000),
                                k_features='parsimonious',
                                forward=False,
                                scoring=ftwo_scorer,
                                cv=KFold(n_splits=5, shuffle=True,random_state=False))
selected_features = sfs.fit(normalizedTrainDF, df_train['label'].values).k_feature_names_
model = LogisticRegression(multi_class='multinomial', solver='lbfgs',max_iter=12000, class_weight={0:2.5,1:1})
model.fit(normalizedTrainDF[np.asarray(selected_features)], df_train['label'].values, sample_weight = weights)
yhat = model.predict(normalizedTestDF[np.asarray(selected_features)])
testy= df_test['label'].values

yhat = model.predict(normalizedTestDF[np.asarray(selected_features)])
testy= df_test['label'].values
yhat0 = model.predict(normalizedTrainDF[np.asarray(selected_features)])
trainy = df_train['label'].values


# Check accuracy on testing data
yhat = model.predict(normalizedTestDF[np.asarray(selected_features)])
testy= df_test['label'].values
yhat0 = model.predict(normalizedTrainDF[np.asarray(selected_features)])
trainy = df_train['label'].values

yhat2 = model.predict(normalizedTestLabDF[np.asarray(selected_features)])
testy2= labtest['label'].values

print('Train Accuracy: '+ str(np.sum(yhat0==trainy)/len(trainy)))
print( fbeta_score(trainy, yhat0, average='weighted', beta=2))

print('Test Accuracy: '+ str(np.sum(yhat==testy)/len(testy)))
print(fbeta_score(testy, yhat, average='weighted', beta=2))

print('Lab Test Accuracy: '+ str(np.sum(yhat==testy)/len(testy)))
f2(testy2, yhat2)

print()
print('similarity with best model')
vsbest(model)
print()

print(selected_features)

Train Accuracy: 0.9835820895522388
0.9835346908245846
Test Accuracy: 0.9868159203980099
0.9867820178659873
Lab Test Accuracy: 0.9868159203980099
true positives 21
false positives 3
precision 0.875
recall 0.9545454545454546
F2 0.9375

similarity with best model
0.7756813417190775

('meanStepDisplacement', 'meanStepTravel', 'shapiroDisplacement', 'meanAngle', 'sdAngle', 'sdVideoMeanStepTravel', 'meanVideoAngle', 'sdVideoAngle')


### SVM

In [344]:
from sklearn import svm
clf = svm.SVC(C=10, kernel = 'sigmoid', degree = 3)
clf = svm.SVC(C=10, kernel = 'poly', degree = 3, probability = True)
model2 = svm.SVC(C=15, kernel = 'poly', degree = 3, class_weight = 'balanced')
clf = CalibratedClassifierCV(model2, method='isotonic', cv=10)
clf.fit(normalizedTrainDF,df_train['label'].values)

CalibratedClassifierCV(cv=10,
                       estimator=SVC(C=15, class_weight='balanced',
                                     kernel='poly'),
                       method='isotonic')

In [346]:
yhat = clf.predict(normalizedTestDF)
testy= df_test['label'].values

yhat0 = clf.predict(normalizedTrainDF)
trainy = df_train['label'].values


yhat2 = clf.predict(normalizedTestLabDF)
testy2= labtest['label'].values

print('Train Accuracy: '+ str(np.sum(yhat0==trainy)/len(trainy)))
print( fbeta_score(trainy, yhat0, average='weighted', beta=2))

print('Test Accuracy: '+ str(np.sum(yhat==testy)/len(testy)))
print(fbeta_score(testy, yhat, average='weighted', beta=2))

print('Lab Test Accuracy: '+ str(np.sum(yhat==testy)/len(testy)))
f2(testy2, yhat2)

print()
print('similarity with best model')
vsbest(model)
print()

print(selected_features)



Train Accuracy: 0.9975953565505804
0.9975952338372116
Test Accuracy: 0.9937810945273632
0.9937809843085593
Lab Test Accuracy: 0.9937810945273632
true positives 22
false positives 2
precision 0.9166666666666666
recall 0.8148148148148148
F2 0.8333333333333334

similarity with best model
0.7756813417190775

('meanStepDisplacement', 'meanStepTravel', 'shapiroDisplacement', 'meanAngle', 'sdAngle', 'sdVideoMeanStepTravel', 'meanVideoAngle', 'sdVideoAngle')


## Decision Trees

In [415]:
def updateW(val):
    for w in range(len(weights)):
        if weights[w] != 0.1:
            weights[w] = val*0.1

updateW(6.9)

In [428]:
from sklearn.ensemble import RandomForestClassifier
np.random.seed(1)

n_estimators = 1000
clf = RandomForestClassifier(n_estimators = n_estimators, criterion = 'gini', class_weight='balanced')
clf.set_params(max_depth = 100).fit(normalizedTrainDF,df_train['label'].values, sample_weight = weights)

RandomForestClassifier(class_weight='balanced', max_depth=100,
                       n_estimators=1000)

In [429]:
yhat = clf.predict(normalizedTestDF)
testy= df_test['label'].values

yhat0 = clf.predict(normalizedTrainDF)
trainy = df_train['label'].values

yhat2 = clf.predict(normalizedTestLabDF)
testy2= labtest['label'].values

print('Train Accuracy: '+ str(np.sum(yhat0==trainy)/len(trainy)))
print( fbeta_score(trainy, yhat0, average='weighted', beta=2))

print('Test Accuracy: '+ str(np.sum(yhat==testy)/len(testy)))
print(fbeta_score(testy, yhat, average='weighted', beta=2))

print('Lab Test Accuracy: '+ str(np.sum(yhat==testy)/len(testy)))
f2(testy2, yhat2)

print()
print('similarity with best model')
vsbest(model)
print()

print(selected_features)

Train Accuracy: 0.9996683250414594
0.9996682992919363
Test Accuracy: 0.9945273631840796
0.994526471931705
Lab Test Accuracy: 0.9945273631840796
true positives 23
false positives 5
precision 0.8214285714285714
recall 0.9583333333333334
F2 0.9274193548387097

similarity with best model
0.7358490566037735

('meanStepDisplacement', 'sdStepTravel', 'shapiroDisplacement', 'meanAngle', 'meanVideoStepTravel', 'meanVideoAngle', 'sdVideoAngle')


## Converting to data for Kaggle Upload.

In [323]:
TESTdf = pd.read_csv("testingData2.csv")
NPTEST = TESTdf.drop(['lab','UID'], axis=1).values
NPTEST = scaler.transform(NPTEST)
normalizedTESTDF= pd.DataFrame(NPTEST, columns = TESTdf.drop(['lab','UID'], axis=1).columns.values)

In [329]:
# RUN FOR STEPWISE REGRESSION
UIDs = TESTdf['UID'].values
preds = model.predict(normalizedTESTDF[np.asarray(selected_features)]).astype(int)

In [272]:
# RUN FOR SVMs AND DECISION TREES
UIDs = TESTdf['UID'].values
preds = clf.predict(normalizedTESTDF).astype(int)

In [274]:
submission = pd.DataFrame({'UID': UIDs, 'label': preds})
submission.to_csv("submission.csv",index=False)

In [275]:
submission

,UID,label
0,lab_19_0,0
1,lab_19_1,0
2,lab_19_10,0
3,lab_19_11,0
4,lab_19_12,1
...,...,...
472,lab_42_5,0
473,lab_42_6,0
474,lab_42_7,1
475,lab_42_8,1


## Historical Comparison

Also good to check we haven't replicated a worse result

In [135]:
baddf = pd.read_csv("submissionbad.csv")
gooddf = pd.read_csv("submissionbest.csv")
predsbad = baddf['label'].values
predsgood = gooddf['label'].values

In [330]:
print("overlap with worst prediction")
print(np.sum(preds==predsbad)/len(preds))

overlap with worst prediction
0.8218029350104822


In [331]:
print("overlap with best prediction")
print(np.sum(preds==predsgood)/len(preds))

overlap with best prediction
0.8071278825995807


In [335]:
def vsbest(model):
    preds = model.predict(normalizedTESTDF[np.asarray(selected_features)]).astype(int)
    print(np.sum(preds==predsgood)/len(preds))

In [ ]:
vals = [0.7425, 0.55397, 0.63186, 0.74453, 0.75172]
for i in vals:
    